In [38]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot  as plt
import re
import seaborn as sns
from matplotlib.colors import LinearSegmentedColormap
from sklearn.utils import resample

In [ ]:
# parameterisation
classic_scene_weight = 1.2
theory_scene_weight = 1.0
real_UAV_weight = 1.5
visual_UAV_weight = 1.0
beta = 0.3

In [ ]:
alo_name='Straight'
file_path = 'Pre-treatment_'+alo_name+'_success_rate.xlsx'  # Replace with your file path
sheet_name = 'Sheet1'  # Replace with your worksheet name
df = pd.read_excel(file_path, sheet_name=sheet_name)
print(df)

                 max_vel=4m/s  Forest  Urban  Random-Angle  Narrow  \
0                0.55kg-UAV 1     0.1    0.2             0     0.1   
1                 0.60kg-EMAX     0.1    0.2             0     0.1   
2   0.68kg-Agile Autonomy DIY     0.1    0.2             0     0.1   
3                0.75kg-UAV 2     0.1    0.2             0     0.1   
4                0.85kg-UAV 3     0.1    0.2             0     0.1   
5                 0.895kg-DJI     0.1    0.2             0     0.1   
6                  0.90kg-DJI     0.1    0.2             0     0.1   
7      0.98kg-EGO Planner DIY     0.1    0.2             0     0.1   
8             1.00kg-SunnySky     0.1    0.2             0     0.1   
9                1.05kg-UAV 4     0.1    0.2             0     0.1   
10               1.20kg-UAV 5     0.1    0.2             0     0.1   
11                1.20kg-JFRC     0.1    0.2             0     0.1   
12                1.40kg-EMAX     0.1    0.2             0     0.1   
13               1.5

In [41]:
# print(df)
df = df.dropna(axis=1, how='all')
scene_num = df.shape[1] - 1
UAV_num = df.shape[0]
print(scene_num)

df_num = df.iloc[0:, 1:]
df_num = df_num.astype(float)
# print(df_num)

7


In [ ]:
# normalisation parameter
w_s_classic = classic_scene_weight / (classic_scene_weight * 3 + theory_scene_weight * (scene_num - 3))
w_s_theory = theory_scene_weight / (classic_scene_weight * 3 + theory_scene_weight * (scene_num - 3))
# print(w_s_classic , w_s_theory)
w_U_real = real_UAV_weight / (real_UAV_weight * (UAV_num / 2) + visual_UAV_weight * (UAV_num / 2))
w_U_visual = visual_UAV_weight / (real_UAV_weight * (UAV_num / 2) + visual_UAV_weight * (UAV_num / 2))
# print(w_U_real , w_U_visual)

In [ ]:
# Constructing the parameter matrix
W_S = np.zeros((UAV_num, scene_num))
W_UAV = np.zeros((UAV_num, scene_num))
S = np.zeros((UAV_num, scene_num))

R_or_not = np.zeros((UAV_num, 1))
R_or_not[1] = R_or_not[5] = R_or_not[6] = R_or_not[8] = R_or_not[11] = R_or_not[12] = R_or_not[14] = R_or_not[16] = R_or_not[18] = R_or_not[20] = R_or_not[22] = R_or_not[24] = R_or_not[26] = R_or_not[27] = R_or_not[28] = R_or_not[31] = R_or_not[33] = R_or_not[35] =1


for i in range(UAV_num):
    for j in range(scene_num):
        if j < 3:
            W_S[i][j] = w_s_classic
        else:
            W_S[i][j] = w_s_theory
            
for i in range(UAV_num):
    if R_or_not[i] == 1:
        for j in range(scene_num):
            W_UAV[i][j] = w_U_real
    else:
        for j in range(scene_num):
            W_UAV[i][j] = w_U_visual
            

for i in range(UAV_num):
    for j in range(scene_num):
        S[i][j] = df_num.iloc[i][j]
        
print(S)

[[0.1 0.2 0.  0.1 0.  0.  0. ]
 [0.1 0.2 0.  0.1 0.  0.  0. ]
 [0.1 0.2 0.  0.1 0.  0.  0. ]
 [0.1 0.2 0.  0.1 0.  0.  0. ]
 [0.1 0.2 0.  0.1 0.  0.  0. ]
 [0.1 0.2 0.  0.1 0.  0.  0. ]
 [0.1 0.2 0.  0.1 0.  0.  0. ]
 [0.1 0.2 0.  0.1 0.  0.  0. ]
 [0.1 0.2 0.  0.1 0.  0.  0. ]
 [0.1 0.2 0.  0.1 0.  0.  0. ]
 [0.1 0.2 0.  0.1 0.  0.  0. ]
 [0.1 0.2 0.  0.1 0.  0.  0. ]
 [0.1 0.2 0.  0.1 0.  0.  0. ]
 [0.1 0.2 0.  0.1 0.  0.  0. ]
 [0.1 0.2 0.  0.1 0.  0.  0. ]
 [0.1 0.2 0.  0.1 0.  0.  0. ]
 [0.1 0.2 0.  0.1 0.  0.  0. ]
 [0.1 0.2 0.  0.1 0.  0.  0. ]
 [0.1 0.2 0.  0.1 0.  0.  0. ]
 [0.1 0.2 0.  0.1 0.  0.  0. ]
 [0.1 0.2 0.  0.1 0.  0.  0. ]
 [0.1 0.2 0.  0.1 0.  0.  0. ]
 [0.1 0.2 0.  0.1 0.  0.  0. ]
 [0.1 0.2 0.  0.1 0.  0.  0. ]
 [0.1 0.2 0.  0.1 0.  0.  0. ]
 [0.1 0.2 0.  0.1 0.  0.  0. ]
 [0.1 0.2 0.  0.1 0.  0.  0. ]
 [0.1 0.2 0.  0.1 0.  0.  0. ]
 [0.1 0.2 0.  0.1 0.  0.  0. ]
 [0.1 0.2 0.  0.1 0.  0.  0. ]
 [0.1 0.2 0.  0.1 0.  0.  0. ]
 [0.1 0.2 0.  0.1 0.  0.  0. ]
 [0.1 0.

C:\Users\Administrator\AppData\Local\Temp\ipykernel_18972\1843311770.py:28: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  S[i][j] = df_num.iloc[i][j]


In [44]:
score_ = sum( sum( W_UAV * W_S * S ) ) / sum( sum( W_UAV * W_S ) )

SCORE = score_ *100
print(SCORE)

VAR = sum( sum( W_UAV * W_S * (S - score_) * (S - score_) ) )

print(VAR)

final_score = SCORE * (1 - beta * VAR/0.122)
print(final_score)

6.052631578947365
0.005547091412742381
5.9700713314324325
